In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code 
    true_positives = K.sum(y_true*y_pred)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def createModel(convFilters, convAndPoolLayers,numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate,L2Rate):
    model = keras.Sequential()
        
    for convAndPoolLayer in range(convAndPoolLayers):
        model.add(keras.layers.Conv2D(convFilters,(3,3), activation='relu',padding='valid'))
        model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
        
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='softmax'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(L2Rate)))
            model.add(keras.layers.Dropout(dropoutRate))
            
    adamOptimizer = keras.optimizers.legacy.Adam(learning_rate=adamLearningRate)       
    model.compile(optimizer="adam",loss=f1_loss, metrics=f1_score)
    return model

In [5]:
def createModelParametersDF(n_convFilters,n_convAndPoolLayers,
                            n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                            dropoutRates,adamLearningRates,L2Rates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_convFilters'] = n_convFilters
    modelParameters['n_convAndPoolLayers'] = n_convAndPoolLayers
    modelParameters['n_FCLayers'] = n_FCLayers
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['n_Epochs'] = n_Epochs
    modelParameters['dropoutRate'] = dropoutRates
    modelParameters['adamLearningRates'] = adamLearningRates
    modelParameters['L2Rates'] = L2Rates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [6]:
def createRangeFromMidpoint(midpoint,range,mandatoryMinimum=1):
    possibleMin = int(midpoint-(range/2))
    possibleMin = max([mandatoryMinimum,possibleMin])
    possibleMax = int(midpoint+(range/2))
    possibleRange = np.arange(possibleMin,possibleMax)
    return possibleRange

In [7]:
def f1_loss(y_true,y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    f1 = (2 * precision * recall) / (precision + recall + K.epsilon())
    return 1 - f1

In [8]:
def generateAdamLearningRate(lastBestLearningRate=None):
    learningRatePower = np.random.random() * (-2 + 4) - 4
    learningRate = np.power(10,learningRatePower)
    if lastBestLearningRate != None:
        learningRate = (learningRate + lastBestLearningRate) / 2
    return learningRate

In [9]:
def generateL2(lastBestL2=None):
    l2Power = np.random.random() * (3 + 2) - 2
    l2 = np.power(10,l2Power)
    if lastBestL2 != None:
        l2 = (l2 + lastBestL2) / 2
    return l2

In [10]:
def main():

    possibleConvFilters = createRangeFromMidpoint(32,32)

    possibleConvAndPoolLayers = createRangeFromMidpoint(3,6)

    possibleNumberOfFCLayers = createRangeFromMidpoint(16,16)
    possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(16,16)

    possibleNumberOfEpochs = createRangeFromMidpoint(20,20)
    trial = 0
    
    train = loadData("training")
    dev,test = loadData("testing",.5)

    choose = np.random.choice

    n_convFilters = []
    n_convAndPoolLayers = []
    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    n_Epochs = []
    dropoutRates = []
    adamLearningRates = []
    L2Rates = []
    trainScores = []
    devScores = []
    
    lastTrialForNotebook = trial + 10
    
    while trial < lastTrialForNotebook:
        convFilters = choose(possibleConvFilters)

        convAndPoolLayers = choose(possibleConvAndPoolLayers)
        convAndPoolLayers = min([convAndPoolLayers,6])

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        dropoutRate = np.random.rand()
        adamLearningRate = generateAdamLearningRate()
        L2Rate = generateL2()
        

        model = createModel(convFilters, convAndPoolLayers,numberOfFCLayers, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
    
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            n_convFilters.append(convFilters)
            n_convAndPoolLayers.append(convAndPoolLayers)
            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            n_Epochs.append(numberOfEpochs)
            adamLearningRates.append(adamLearningRate)
            dropoutRates.append(dropoutRate)
            L2Rates.append(L2Rate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createModelParametersDF([convFilters],[convAndPoolLayers],
                                                  [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                                  [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
            display(failedTrial)
            
    modelParametersDF = createModelParametersDF(n_convFilters,n_convAndPoolLayers,
                                                n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
    display(modelParametersDF.sort_values(by='trainScore', ascending=False))
        
    
    

In [11]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-06-15 13:17:00.959929: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-15 13:17:00.960070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-15 13:17:01.470593: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 14s 173ms/step - loss: 0.3519 - f1_score: 0.6478
devScore


2024-06-15 13:27:04.950980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-06-15 13:27:04.951137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 2s 125ms/step - loss: 0.3657 - f1_score: 0.6277
concluding trial  0

trainScore
75/75 [==============================] - 10s 123ms/step - loss: 0.3521 - f1_score: 0.6480
devScore
10/10 [==============================] - 1s 82ms/step - loss: 0.3692 - f1_score: 0.6244
concluding trial  1
redoing trial 2. Model was 101.66500473022461MB.


,n_convFilters,n_convAndPoolLayers,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
0,25,1,9,22,17,0.086116,0.000237,0.058411,NaN,NaN



trainScore
75/75 [==============================] - 13s 160ms/step - loss: 0.3505 - f1_score: 0.6493
devScore
10/10 [==============================] - 2s 110ms/step - loss: 0.3664 - f1_score: 0.6384
concluding trial  2

trainScore
75/75 [==============================] - 10s 122ms/step - loss: 0.3605 - f1_score: 0.6483
devScore
10/10 [==============================] - 1s 69ms/step - loss: 0.3838 - f1_score: 0.6196
concluding trial  3

trainScore
75/75 [==============================] - 9s 109ms/step - loss: 3.1915 - f1_score: 0.6484
devScore
10/10 [==============================] - 1s 69ms/step - loss: 3.2070 - f1_score: 0.6320
concluding trial  4

trainScore
75/75 [==============================] - 12s 155ms/step - loss: 0.3509 - f1_score: 0.6491
devScore
10/10 [==============================] - 1s 80ms/step - loss: 0.3644 - f1_score: 0.6451
concluding trial  5

trainScore
75/75 [==============================] - 13s 164ms/step - loss: 0.3491 - f1_score: 0.6510
devScore
10/10 [======

,n_convFilters,n_convAndPoolLayers,n_FCLayers,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
6,37,5,17,17,27,0.893147,0.000555,143.215169,0.650965,0.638469
2,40,5,11,20,28,0.915330,0.000516,117.000360,0.649271,0.638418
7,30,5,10,16,13,0.755836,0.001247,0.212155,0.649250,0.637643
5,36,5,9,8,27,0.297469,0.008205,550.025688,0.649083,0.645058
9,36,3,11,15,12,0.769192,0.000120,398.657303,0.648749,0.645991
8,20,4,10,11,24,0.732762,0.000544,0.019914,0.648723,0.650836
4,32,2,8,15,10,0.866173,0.000110,31.591589,0.648439,0.631977
3,17,5,10,10,17,0.609452,0.004888,7.198328,0.648290,0.619625
1,25,5,15,11,28,0.316866,0.000251,0.079393,0.647964,0.624442
0,41,3,11,15,16,0.417306,0.001020,0.108814,0.647825,0.627677
